In [22]:
import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

warnings.filterwarnings("ignore")

dir_f = '../input/'
dir_list = os.listdir(dir_f)
#print(dir_list)

df = pd.read_csv('../input/train.csv')
df = df.drop('id', axis=1)
df.head()

In [23]:
print('# unique',len(df.species.unique()))
print('# rows', len(df.index))

**K Nearest Neighbors (p=1)**

In [24]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

target = 'species'
X = df.drop(target,axis=1)
y = df[target]

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=42)

knn = KNeighborsClassifier(n_neighbors=1, p=1)
knn.fit(X_train, y_train)
acc = knn.score(X_val, y_val)
print('accuracy', acc)

**Neural Network**

In [26]:
# Import necessary modules
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.optimizers import SGD, Adam
from sklearn.preprocessing import LabelEncoder

lb_make = LabelEncoder()
target = lb_make.fit_transform(df['species'])
target = to_categorical(target)
X = df.drop('species',axis=1)

def simple_NN(input_shape, nodes_per=[60], hidden=0, out=2, act_out='softmax', act_hid='relu', drop=True, d_rate=0.1):
  """Generate a keras neural network with arbitrary number of hidden layers each having the same number of nodes"""
  model = Sequential()
  model.add(Dense(nodes_per[0],activation=act_hid,input_shape=input_shape))
  if drop == True:
      model.add(Dropout(d_rate))
  try:
    if hidden != 0:
      for i,j in zip(range(hidden), nodes_per[1:]):
          model.add(Dense(j,activation=act_hid))
          if drop == True:
              model.add(Dropout(d_rate))
    model.add(Dense(out,activation=act_out))
    return(model)
  except:
    print('Error in generating hidden layers')
                  
model = simple_NN((X.shape[1],), nodes_per=[60], hidden=0, out=len(df.species.unique()), drop=True)

model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])
early_stopping_monitor = EarlyStopping(patience=5)                  
history = model.fit(X,target,epochs=1000, validation_split=0.1, callbacks=[early_stopping_monitor], verbose=False)
print('model trained')

**Plot Model Statistics**

In [27]:
def plt_perf(name, p_loss=False, p_acc=False, val=False, size=(15,9), save=False):
  """Plot model statistics for keras models"""
  if p_loss or p_acc:
    if p_loss:
      plt.figure(figsize = size)
      plt.title('Loss')
      plt.plot(name.history['loss'], 'b', label='loss')
      if val:
        plt.plot(name.history['val_loss'], 'r', label='val_loss')
      plt.xlabel('Epochs')
      plt.ylabel('Value')
      plt.legend()
      plt.show()
      if save:
        plt.savefig('loss.png')
    if p_acc:
      plt.figure(figsize = size)
      plt.title('Accuracy')
      plt.plot(name.history['acc'], 'b', label='acc')
      if val:
        plt.plot(name.history['val_acc'], 'r', label='val_acc')
      plt.xlabel('Epochs')
      plt.ylabel('Value')
      plt.legend()
      plt.show()
      if save:
        plt.savefig('acc.png')
  else:
    print('No plotting since all parameters set to false.')
    
plt_perf(history, p_loss=True, p_acc=True, val=True)